# Interactive visualizer
Using [Interactive visualizers](https://kaolin.readthedocs.io/en/latest/modules/kaolin.visualize.html) you can bring your own renderer and connect it to the visualizer.

The main condition is that the renderer have to take a [Camera](https://kaolin.readthedocs.io/en/latest/modules/kaolin.render.camera.camera.html#kaolin-render-camera-camera) as input.

In [1]:
import glob
import math
import copy

import torch
import numpy as np

import kaolin as kal

import nvdiffrast
glctx = nvdiffrast.torch.RasterizeGLContext(False, device='cuda')

## Load Mesh information

In [2]:
# Here replace the "root" by the path where you are storing shapenet
ds = kal.io.shapenet.ShapeNetV2(root='/data/ShapeNetCore.v2/',
                                categories=['car'],
                                train=True, split=1.,
                                with_materials=True,
                                output_dict=True)
mesh = ds[0]['mesh']

### Uncomment to load a specific obj
# mesh = kal.io.obj.import_mesh('path/to/obj', with_materials=True)

# Normalize the data between [-0.5, 0.5]
vertices = mesh.vertices.unsqueeze(0).cuda()
vertices_min = vertices.min(dim=1, keepdims=True)[0]
vertices_max = vertices.max(dim=1, keepdims=True)[0]
vertices -= (vertices_max + vertices_min) / 2.
vertices /= (vertices_max - vertices_min).max()
faces = mesh.faces.cuda()

# Here we are preprocessing the materials, assigning faces to materials and
# using single diffuse color as backup when map doesn't exist (and face_uvs_idx == -1)
uvs = torch.nn.functional.pad(mesh.uvs.unsqueeze(0).cuda(), (0, 0, 0, 1)) % 1.
face_uvs_idx = mesh.face_uvs_idx.cuda()
materials_order = mesh.materials_order
diffuse_maps = [m['map_Kd'].permute(2, 0, 1).unsqueeze(0).cuda().float() / 255. if 'map_Kd' in m else
                m['Kd'].reshape(1, 3, 1, 1).cuda()
                for m in mesh.materials]
specular_maps = [m['map_Ks'].permute(2, 0, 1).unsqueeze(0).cuda().float() / 255. if 'map_Ks' in m else
                 m['Ks'].reshape(1, 3, 1, 1).cuda()
                 for m in mesh.materials]

nb_faces = faces.shape[0]

num_consecutive_materials = \
    torch.cat([
        materials_order[1:, 1],
        torch.LongTensor([nb_faces])
    ], dim=0)- materials_order[:, 1]

face_material_idx = kal.ops.batch.tile_to_packed(
    materials_order[:, 0],
    num_consecutive_materials
).cuda().squeeze(-1)
mask = face_uvs_idx == -1
face_uvs_idx[mask] = uvs.shape[1] - 1
face_vertices = kal.ops.mesh.index_vertices_by_faces(vertices, faces)
face_world_normals = kal.ops.mesh.face_normals(face_vertices, unit=True)

## Instantiate a camera

With the general constructor `Camera.from_args()` the underlying constructors are `CameraExtrinsics.from_lookat()` and `PinholeIntrinsics.from_fov` we will use this camera as a starting point for the visualizers.

In [3]:
camera = kal.render.camera.Camera.from_args(eye=torch.tensor([2., 1., 1.], device='cuda'),
                                            at=torch.tensor([0., 0., 0.]),
                                            up=torch.tensor([1., 1., 1.]),
                                            fov=math.pi * 45 / 180,
                                            width=512, height=512, device='cuda')

## Rendering a mesh

Here we are rendering the loaded mesh with [nvdiffrast](https://github.com/NVlabs/nvdiffrast) using the camera object created above and use both diffuse and specular reflectance for lighting.

For more information on lighting in Kaolin see [diffuse](./diffuse_lighting.ipynb) and [specular](./sg_specular_lighting.ipynb) tutorials and the [documentation](https://kaolin.readthedocs.io/en/latest/modules/kaolin.render.lighting.html).

In [4]:
# Those are the parameters used to define the Spherical gaussian
azimuth = torch.zeros((1,), device='cuda')
elevation = torch.full((1,), math.pi / 3., device='cuda')
amplitude = torch.full((1, 3), 3., device='cuda')
sharpness = torch.full((1,), 5., device='cuda')
# We will use this variable to enable / disable specular reflectance
global apply_specular
apply_specular = True

def generate_pinhole_rays_dir(camera, height, width, device='cuda'):
    """Generate centered grid.
    
    This is a utility function for specular reflectance with spherical gaussian.
    """
    pixel_y, pixel_x = torch.meshgrid(
        torch.arange(height, device=device),
        torch.arange(width, device=device),
        indexing='ij'
    )
    pixel_x = pixel_x + 0.5  # scale and add bias to pixel center
    pixel_y = pixel_y + 0.5  # scale and add bias to pixel center

    # Account for principal point (offsets from the center)
    pixel_x = pixel_x - camera.x0
    pixel_y = pixel_y + camera.y0

    # pixel values are now in range [-1, 1], both tensors are of shape res_y x res_x
    # Convert to NDC
    pixel_x = 2 * (pixel_x / width) - 1.0
    pixel_y = 2 * (pixel_y / height) - 1.0

    ray_dir = torch.stack((pixel_x * camera.tan_half_fov(kal.render.camera.intrinsics.CameraFOV.HORIZONTAL),
                           -pixel_y * camera.tan_half_fov(kal.render.camera.intrinsics.CameraFOV.VERTICAL),
                           -torch.ones_like(pixel_x)), dim=-1)

    ray_dir = ray_dir.reshape(-1, 3)    # Flatten grid rays to 1D array
    ray_orig = torch.zeros_like(ray_dir)

    # Transform from camera to world coordinates
    ray_orig, ray_dir = camera.extrinsics.inv_transform_rays(ray_orig, ray_dir)
    ray_dir /= torch.linalg.norm(ray_dir, dim=-1, keepdim=True)

    return ray_dir[0].reshape(1, height, width, 3)


def base_render(camera, height, width):
    """Base function for rendering using separate height and width"""
    transformed_vertices = camera.transform(vertices)
    face_vertices_camera = kal.ops.mesh.index_vertices_by_faces(
        transformed_vertices, faces)
    face_normals_z = kal.ops.mesh.face_normals(
        face_vertices_camera,
        unit=True
    )[..., -1:].contiguous()
    # Create a fake W (See nvdiffrast documentation)
    pos = torch.nn.functional.pad(
        transformed_vertices, (0, 1), mode='constant', value=1.
    ).contiguous()
    rast = nvdiffrast.torch.rasterize(
        glctx, pos, faces.int(), (height, width), grad_db=False)
    hard_mask = rast[0][:, :, :, -1:] != 0
    face_idx = (rast[0][..., -1].long() - 1).contiguous()

    uv_map = nvdiffrast.torch.interpolate(
        uvs, rast[0], face_uvs_idx.int())[0]

    im_world_normals = face_world_normals.reshape(-1, 3)[face_idx]
    im_cam_normals = face_normals_z.reshape(-1, 1)[face_idx]
    im_world_normals = im_world_normals * torch.sign(im_cam_normals)
    albedo = torch.zeros(
        (1, height, width, 3),
        dtype=torch.float, device='cuda'
    )
    spec_albedo = torch.zeros(
        (1, height, width, 3),
        dtype=torch.float, device='cuda'
    )
    # Obj meshes can be composed of multiple materials
    # so at rendering we need to interpolate from corresponding materials
    im_material_idx = face_material_idx[face_idx]
    im_material_idx[face_idx == -1] = -1

    for i, material in enumerate(diffuse_maps):
        mask = im_material_idx == i
        mask_idx = torch.nonzero(mask, as_tuple=False)
        _texcoords = uv_map[mask] * 2. - 1.
        _texcoords[:, 1] = -_texcoords[:, 1]
        pixel_val = torch.nn.functional.grid_sample(
            diffuse_maps[i], _texcoords.reshape(1, 1, -1, 2),
            mode='bilinear', align_corners=False,
            padding_mode='border')
        albedo[mask] = pixel_val[0, :, 0].permute(1, 0)
        pixel_val = torch.nn.functional.grid_sample(
            specular_maps[i], _texcoords.reshape(1, 1, -1, 2),
            mode='bilinear', align_corners=False,
            padding_mode='border')
        spec_albedo[mask] = pixel_val[0, :, 0].permute(1, 0)
    img = torch.zeros((1, height, width, 3),
                      dtype=torch.float, device='cuda')
    sg_x, sg_y, sg_z = kal.ops.coords.spherical2cartesian(azimuth, elevation)
    directions = torch.stack(
        [sg_x, sg_z, sg_y],
        dim=-1
    )
    im_world_normals = im_world_normals[hard_mask.squeeze(-1)]
    diffuse_effect = kal.render.lighting.sg_diffuse_inner_product(
        amplitude, directions, sharpness,
        im_world_normals,
        albedo[hard_mask.squeeze(-1)]
    )
    img[hard_mask.squeeze(-1)] = diffuse_effect
    global apply_specular
    if apply_specular:
        rays_d = generate_pinhole_rays_dir(camera, height, width)
        specular_effect = kal.render.lighting.sg_warp_specular_term(
            amplitude, directions, sharpness,
            im_world_normals,
            torch.full((im_world_normals.shape[0],), 0.5, device='cuda'),
            -rays_d[hard_mask.squeeze(-1)],
            spec_albedo[hard_mask.squeeze(-1)]
        )
        img[hard_mask.squeeze(-1)] += specular_effect

    # Need to flip the image because opengl
    return (torch.flip(torch.clamp(
        img * hard_mask, 0., 1.
    )[0], dims=(0,)) * 255.).to(torch.uint8)

def render(camera):
    """Render using camera dimension.
    
    This is the main function provided to the interactive visualizer
    """
    return base_render(camera, camera.height, camera.width)

def lowres_render(camera):
    """Render with lower dimension.
    
    This function will be used as a "fast" rendering used when the mouse is moving to avoid slow down.
    """
    return base_render(camera, int(camera.height / 4), int(camera.width / 4))

## Turntable visualizer
This is a simple visualizer useful to inspect a small object.

You can move around with the mouse (left button) and zoom with the mouse wheel.
See the [documentation](https://kaolin.readthedocs.io/en/latest/modules/kaolin.visualize.html#kaolin.visualize.IpyTurntableVisualizer) to customize the sensitivity.

In [5]:
visualizer = kal.visualize.IpyTurntableVisualizer(
    512, 512, copy.deepcopy(camera), render,
    fast_render=lowres_render, max_fps=24, world_up_axis=1)
visualizer.show()

Canvas(height=512, width=512)

Output()

## First person visualizer
This is a visualizer useful to inspect details on an object, or a big scene.

You can move the orientation of the camera with the mouse left button, move the camera around with the mouse right button or
the keys 'i' (up), 'k' (down), 'j' (left), 'l' (right), 'o' (forward), 'u' (backward)

See the [documentation](https://kaolin.readthedocs.io/en/latest/modules/kaolin.visualize.html#kaolin.visualize.IpyFirstPersonVisualizer) to customize the sensitivity and keys.

--------------------
*Note: camera are mutable in the visualizer. If you want to keep track of the camera position you can remove the `copy.deepcopy` on camera argument or you can check `visualizer.camera`*

In [6]:
visualizer = kal.visualize.IpyFirstPersonVisualizer(
    512, 512, copy.deepcopy(camera), render, fast_render=lowres_render,
    max_fps=24, world_up=torch.tensor([0., 1., 0.], device='cuda'))
visualizer.show()

Canvas(height=512, width=512)

Output()

# Adding events and other widgets

The visualizer is modular.
Here we will add:
* sliders to control the spherical gaussian parameters (see [ipywidgets tutorial](https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html) for more info).
* A key event to 'space' to enable / disable specular reflectance (see [ipyevents documentation](https://github.com/mwcraig/ipyevents/blob/main/docs/events.ipynb)) to see all the events that can be caught.

In general if you want to modify the rendering function you can use global variables or make a class (with the rendering function being a method)

-------------
More info on spherical gaussians parameters on this [tutorial]((./sg_specular_lighting.ipynb)
and [documentation](https://kaolin.readthedocs.io/en/latest/modules/kaolin.render.lighting.html)

In [7]:
from ipywidgets import interactive, HBox, FloatSlider

def additional_event_handler(visualizer, event):
    """Event handler to be provided to Kaolin's visualizer"""
    with visualizer.out: # This is for catching print and errors
        if event['type'] == 'keydown' and event['key'] == ' ':
            global apply_specular
            apply_specular = not apply_specular
            visualizer.render_update()
            return False
        return True

visualizer = kal.visualize.IpyTurntableVisualizer(
    512, 512, copy.deepcopy(camera), render,
    fast_render=lowres_render, max_fps=24,
    additional_event_handler=additional_event_handler,
    additional_watched_events=['keydown'] # We need to now watch for key press event
)
# we don't call visualizer.show() here

def sliders_callback(new_elevation, new_azimuth, new_amplitude, new_sharpness):
    """ipywidgets sliders callback"""
    with visualizer.out: # This is in case of bug
        elevation[:] = new_elevation
        azimuth[:] = new_azimuth
        amplitude[:] = new_amplitude
        sharpness[:] = new_sharpness
        # this is how we request a new update
        visualizer.render_update()
        
elevation_slider = FloatSlider(
    value=0.,
    min=-math.pi / 2.,
    max=math.pi / 2.,
    step=0.1,
    description='Elevation:',
    continuous_update=True,
    readout=True,
    readout_format='.1f',
)

azimuth_slider = FloatSlider(
    value=0.,
    min=-math.pi,
    max=math.pi,
    step=0.1,
    description='Azimuth:',
    continuous_update=True,
    readout=True,
    readout_format='.1f',
)

amplitude_slider = FloatSlider(
    value=5.,
    min=0.1,
    max=20.,
    step=0.1,
    description='Amplitude:\n',
    continuous_update=True,
    readout=True,
    readout_format='.1f',
)

sharpness_slider = FloatSlider(
    value=5.,
    min=0.1,
    max=20.,
    step=0.1,
    description='Sharpness:\n',
    continuous_update=True,
    readout=True,
    readout_format='.1f',
)

interactive_slider = interactive(
    sliders_callback,
    new_elevation=elevation_slider,
    new_azimuth=azimuth_slider,
    new_amplitude=amplitude_slider,
    new_sharpness=sharpness_slider
)

# We combine all the widgets and the visualizer canvas and output in a single display
full_output = HBox([visualizer.canvas, interactive_slider])
display(full_output, visualizer.out)

Output()